In [1]:
# 통계분석 라이브러리
# 선형회귀, 시계열 분석, 분산 분석, 로지스틱 회귀같은 알고리즘을 통해 
# 모델을 구현하는데 사용됨
!pip install statsmodels

#!pip install scikit-learn


   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
    --------------------------------------- 0.2/9.8 MB 5.8 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/9.8 MB 8.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/9.8 MB 9.7 MB/s eta 0:00:01
   ------- -------------------------------- 1.9/9.8 MB 10.3 MB/s eta 0:00:01
   ---------- ----------------------------- 2.6/9.8 MB 10.9 MB/s eta 0:00:01
   ------------ --------------------------- 3.1/9.8 MB 10.9 MB/s eta 0:00:01
   -------------- ------------------------- 3.5/9.8 MB 10.6 MB/s eta 0:00:01
   ---------------- ----------------------- 3.9/9.8 MB 10.9 MB/s eta 0:00:01
   ------------------ --------------------- 4.5/9.8 MB 11.1 MB/s eta 0:00:01
   -------------------- ------------------- 5.1/9.8 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 5.7/9.8 MB 11.3 MB/s eta 0:00:01
   ------------------------- -------------- 6.3/9.8 MB 11.1 MB/s eta 0:00:01
   -------

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from matplotlib import font_manager, rc

# 한글 폰트
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False


In [4]:
df_origin_data = pd.read_csv('temperature.csv')

In [6]:
df_origin_data.head() # 상위 5개 가져와라

df_origin_data.describe() # 통계치

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
count,23008.000000,23008.000000,23007.000000,9150.000000,23011.000000,23007.000000,22893.000000,18149.000000,22645.000000,23011.000000
mean,17.071714,8.451960,8.619277,9.593683,65.202851,2.380993,5.858826,11.933170,48.653526,12.415419
std,10.714471,10.578285,2.907096,21.966135,14.549077,0.947595,3.816941,6.419122,31.662321,10.489515
min,-13.600000,-20.200000,1.000000,0.000000,17.900000,0.100000,0.000000,0.000000,0.000000,-16.400000
25%,7.800000,-0.300000,6.600000,0.100000,54.900000,1.700000,2.200000,7.000000,17.800000,3.400000
50%,18.900000,9.200000,8.600000,1.400000,65.500000,2.200000,6.600000,11.220000,55.700000,13.800000
75%,26.400000,17.900000,10.600000,8.500000,75.800000,2.900000,9.000000,16.620000,78.000000,21.800000
max,39.600000,30.300000,19.600000,332.800000,99.800000,7.800000,13.700000,33.480000,96.900000,33.700000


In [9]:
# 시계열 데이터: 시간 순서대로 관측된 데이터 (날짜정보를 가공해야함)

df_origin_data.info()

# Name: 일시, Length: 23011, dtype: datetime64[ns]
df_origin_data['일시'] = pd.to_datetime(df_origin_data['일시'])

# '일시'라는 컬럼을 index로 쓰겠다. (inplace=True: 이걸 이제 원본데이터로 쓰겠다)
df_origin_data.set_index('일시', inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      23011 non-null  object 
 1   최고기온    23008 non-null  float64
 2   최저기온    23008 non-null  float64
 3   일교차     23007 non-null  float64
 4   강수량     9150 non-null   float64
 5   평균습도    23011 non-null  float64
 6   평균풍속    23007 non-null  float64
 7   일조합     22893 non-null  float64
 8   일사합     18149 non-null  float64
 9   일조율     22645 non-null  float64
 10  평균기온    23011 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.9+ MB


0       1960-01-01
1       1960-01-02
2       1960-01-03
3       1960-01-04
4       1960-01-05
           ...    
23006   2022-12-27
23007   2022-12-28
23008   2022-12-29
23009   2022-12-30
23010   2022-12-31
Name: 일시, Length: 23011, dtype: datetime64[ns]

In [11]:
df_origin_data.head()

# 데이터의 시간 간격을 지정한다. (D로 설정하면 단위가 하루, 근데 이 때 데이터 중 빠진 값 있으면 안됨)
# 현재 우리가 가진 데이터는 하루도 빠짐없이 모든 데이터를 가지고 있으므로 'D'를 설정한다. (연 단위면 'y', 월 단위면 'm')
# 만약 달의 마지막날 기준으로 데이터가 저장되어 있다면 'M'
# 달의 시작일로 저장되어 있다면 'MS' ex) 1월1일, 2월1일, 3월1일 ...
# 연의 마지막 날 기준이면 'Y'
# 연의 시작일 기준이면 'YS'를 사용한다.
df_origin_data.index.freq = 'D'

In [12]:
# 결측치 구경
df_data = df_origin_data

df_data.isna().sum()

일시          0
최고기온        3
최저기온        3
일교차         4
강수량     13861
평균습도        0
평균풍속        4
일조합       118
일사합      4862
일조율       366
평균기온        0
dtype: int64

In [18]:
# 결측치 상세보기

# 최고 기온
#s_isna = df_data['최고기온'].isna()
#print(df_data[s_isna])

# 최저 기온
#s_isna = df_data['최저기온'].isna()
#print(df_data[s_isna])

df_data.columns

for col in df_data.columns:
    print(df_data[col].isna().value_counts())
    print()
    s_isna = df_data[col].isna()
    print(df_data[s_isna])
    print("--------------------------------------------------")

일시
False    23011
Name: count, dtype: int64

Empty DataFrame
Columns: [일시, 최고기온, 최저기온, 일교차, 강수량, 평균습도, 평균풍속, 일조합, 일사합, 일조율, 평균기온]
Index: []
--------------------------------------------------
최고기온
False    23008
True         3
Name: count, dtype: int64

               일시  최고기온  최저기온  일교차  강수량  평균습도  평균풍속  일조합   일사합   일조율  평균기온
2606   1967-02-19   NaN   NaN  NaN  NaN  62.0   1.8  9.5   NaN  93.1  -1.7
5037   1973-10-16   NaN   NaN  NaN  0.4  74.0   1.8  3.5  9.24  29.7  12.3
21104  2017-10-12   NaN   8.8  NaN  NaN  71.0   2.0  NaN  2.23   0.0  11.4
--------------------------------------------------
최저기온
False    23008
True         3
Name: count, dtype: int64

               일시  최고기온  최저기온  일교차    강수량  평균습도  평균풍속  일조합   일사합   일조율  평균기온
2606   1967-02-19   NaN   NaN  NaN    NaN  62.0   1.8  9.5   NaN  93.1  -1.7
5037   1973-10-16   NaN   NaN  NaN    0.4  74.0   1.8  3.5  9.24  29.7  12.3
22865  2022-08-08  28.4   NaN  NaN  129.6  91.6   3.3  NaN   NaN   0.0  26.8
--------------------------